# ShapeNet

## Dataset

In [9]:
import torch
from torch import nn
from torch import optim
import os
import os.path as osp

import wandb
import pytorch_lightning as pl
import torch_geometric.transforms as T
from torch_geometric.datasets import ShapeNet
from torch_geometric.loader import DataLoader

from pytorch_lightning.loggers import WandbLogger

In [10]:
category = 'Airplane'  # Pass in `None` to train on all categories.
path = osp.realpath(osp.join('..', 'data', 'ShapeNet'))

pre_transform, transform = T.NormalizeScale(), T.FixedPoints(128)
train_dataset = ShapeNet(path, category, split='train', transform=transform, pre_transform=pre_transform)
valid_dataset = ShapeNet(path, category, split='val', transform=transform, pre_transform=pre_transform)
test_dataset = ShapeNet(path, category, split='test', transform=transform, pre_transform=pre_transform)

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=6)
valid_loader = DataLoader(valid_dataset, batch_size=4, shuffle=False, num_workers=6)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False, num_workers=6)

In [11]:
data = train_dataset.get(0)

In [12]:
from hpcs.utils.viz import plot_cloud

In [13]:
# data.x contains points features extracted after having applied the transform
# data.pos contains the points coordinates
plotter = plot_cloud(data.x.numpy(), scalars=data.y.numpy(), point_size=3.0, notebook=True)

## Model

In [30]:
from train_3Dhyperbolic import model

ImportError: cannot import name 'model' from 'train_3Dhyperbolic' (C:\Users\olvcp\PycharmProjects\HPCS\train_3Dhyperbolic.py)

In [27]:
best_model = wandb.restore('model.h5', run_path='pierreoo/HPCS/runs/pmcouv1r')
best_model.load_from_checkpoint('model.h5', run_path='pierreoo/HPCS/runs/pmcouv1r')

AttributeError: '_io.TextIOWrapper' object has no attribute 'load_from_checkpoint'

In [23]:
import pytorch_lightning as pl
trainer = pl.Trainer()

C:\Users\olvcp\anaconda3\envs\hyperbolic\lib\site-packages\pytorch_lightning\loops\utilities.py:92: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  rank_zero_warn(
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [25]:
print(best_model)

<_io.TextIOWrapper name='C:\\Users\\olvcp\\PycharmProjects\\HPCS\\notebooks\\model.h5' mode='r' encoding='cp1252'>


In [24]:
results = trainer.test(best_model, test_loader)

TypeError: Unwrapping the module did not yield a `LightningModule`, got <class '_io.TextIOWrapper'> instead.

In [ ]:
from hpcs.utils.viz import plot_clustering

## Define a Pytorch Lightning module

In [ ]:
class SimpleModel(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.nn = nn.Sequential(
            nn.Linear(3, 64),
            nn.ReLU(),
            nn.Linear(64, 4)
        )
        
    def forward(self, x):
        y_pred = self.nn(x)
        return y_pred
    
    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=1e-3)
        return optimizer
    
    def training_step(self, batch, batch_idx):
        x, y = batch.x, batch.y
        y_hat = self(x)
        loss = nn.functional.cross_entropy(y_hat, y)
        self.log("train/loss", loss)
        return loss
        

In [ ]:
simpl_mod = SimpleModel()

## Training the model

In [ ]:
wandb_logger = WandbLogger(name='SegTest',save_dir='../logs/')

In [ ]:
trainer = pl.Trainer(limit_train_batches=10, max_epochs=10,logger=wandb_logger)

In [ ]:
trainer.fit(model=simpl_mod, train_dataloaders=train_loader)